In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import csv
import os
import matplotlib
import keras
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow_decision_forests as tfdf
import datetime
from sklearn.preprocessing import RobustScaler



2023-11-24 23:07:42.155329: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 23:07:42.155404: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 23:07:42.155431: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 23:07:42.162382: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 23:07:42.965008: W tensorflow/compiler/

In [2]:
print(device_lib.list_local_devices() )
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12871443006622148023
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6680010752
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18246988200251689437
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


2023-11-24 23:07:44.349758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-24 23:07:44.373894: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-24 23:07:44.373955: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-24 23:07:44.501857: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-24 23:07:44.501999: I tensorflow/compile

In [21]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sub = pd.read_csv("sample_submission.csv")


In [22]:
scl = MinMaxScaler()
scl_rb = RobustScaler()

In [23]:
df_tfdf = train.drop(["Name", "PassengerId"], axis=1)
df_test_tfdf = test.drop(["Name", "PassengerId"], axis=1)

In [24]:
df_tfdf["CryoSleep"] = df_tfdf["CryoSleep"].astype(float)
df_tfdf["VIP"] = df_tfdf["VIP"].astype(float)
df_tfdf["Transported"] = df_tfdf["Transported"].astype(float)
df_tfdf["RoomService"].fillna(0, inplace=True)
df_tfdf["FoodCourt"].fillna(0, inplace=True)
df_tfdf["ShoppingMall"].fillna(0, inplace=True)
df_tfdf["CryoSleep"].fillna(0, inplace=True)
df_tfdf["Spa"].fillna(0, inplace=True)
df_tfdf["VRDeck"].fillna(0, inplace=True)
df_tfdf["CryoSleep"].fillna(0, inplace=True)
df_tfdf["VIP"].fillna(0, inplace=True)
df_tfdf["HomePlanet"].fillna("Earth", inplace=True)
df_tfdf["Destination"].fillna("TRAPPIST-1e", inplace=True)
df_tfdf["CryoSleep"].fillna(0, inplace=True)
df_tfdf[["Deck", "Cabin_num", "Side"]] = df_tfdf["Cabin"].str.split("/", expand=True)
df_tfdf = df_tfdf.drop(["Cabin"], axis=1)

In [25]:
sl_tfdf_rs = scl_rb.fit_transform(df_tfdf[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]])
df_sl_rs = pd.DataFrame(sl_tfdf_rs, index=df_tfdf.index, columns=["sl_RS", "sl_FC", "sl_SM", "sl_SPA", "sl_VR"])
df_sl_tfdf = pd.concat([df_tfdf, df_sl_rs], axis=1)
df_sl_tfdf = df_sl_tfdf.drop(["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"], axis=1)


In [26]:
df_test_tfdf["CryoSleep"] = df_test_tfdf["CryoSleep"].astype(float)
df_test_tfdf["VIP"] = df_test_tfdf["VIP"].astype(float)
df_test_tfdf["RoomService"].fillna(0, inplace=True)
df_test_tfdf["Cabin"].fillna(0, inplace=True)
df_test_tfdf["FoodCourt"].fillna(0, inplace=True)
df_test_tfdf["ShoppingMall"].fillna(0, inplace=True)
df_test_tfdf["Spa"].fillna(0, inplace=True)
df_test_tfdf["VRDeck"].fillna(0, inplace=True)
df_test_tfdf["CryoSleep"].fillna(0, inplace=True)
df_test_tfdf["VIP"].fillna(0, inplace=True)
df_test_tfdf["HomePlanet"].fillna("Earth", inplace=True)
df_test_tfdf["Destination"].fillna("TRAPPIST-1e", inplace=True)
df_test_tfdf["CryoSleep"].fillna(0, inplace=True)
df_test_tfdf[["Deck", "Cabin_num", "Side"]] = df_test_tfdf["Cabin"].str.split("/", expand=True)
df_test_tfdf = df_test_tfdf.drop(["Cabin"], axis=1)
sl_test_tfdf_rs = scl_rb.fit_transform(df_test_tfdf[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]])
df_test_sl_rs = pd.DataFrame(sl_test_tfdf_rs, index=df_test_tfdf.index, columns=["sl_RS", "sl_FC", "sl_SM", "sl_SPA", "sl_VR"])
df_test_sl_tfdf = pd.concat([df_test_tfdf, df_test_sl_rs], axis=1)
df_test_sl_tfdf = df_test_sl_tfdf.drop(["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"], axis=1)

In [38]:
"""
iqr = df_no_nm_cb["RoomService"].quantile(0.75) - df_no_nm_cb["RoomService"].quantile(0.25)
df_no_nm_cb["RoomService_iqr75"] = df_no_nm_cb["RoomService"]>=(df_no_nm_cb["RoomService"].quantile(0.75) + (iqr*1.5))
df_no_nm_cb["FoodCourt_iqr75"] = df_no_nm_cb["FoodCourt"]>=(df_no_nm_cb["FoodCourt"].quantile(0.75) + (iqr*1.5))
df_no_nm_cb["ShoppingMall_iqr75"] = df_no_nm_cb["ShoppingMall"]>=(df_no_nm_cb["ShoppingMall"].quantile(0.75) + (iqr*1.5))
df_no_nm_cb["Spa_iqr75"] = df_no_nm_cb["Spa"]>=(df_no_nm_cb["Spa"].quantile(0.75) + (iqr*1.5))
df_no_nm_cb["VRDeck_iqr75"] = df_no_nm_cb["VRDeck"]>=(df_no_nm_cb["VRDeck"].quantile(0.75) + (iqr*1.5))

df_no_nm_cb["RoomService_iqr75"] =df_no_nm_cb["RoomService_iqr75"].astype(int)
df_no_nm_cb["FoodCourt_iqr75"] =df_no_nm_cb["FoodCourt_iqr75"].astype(int)
df_no_nm_cb["ShoppingMall_iqr75"] = df_no_nm_cb["ShoppingMall_iqr75"].astype(int)
df_no_nm_cb["Spa_iqr75"] = df_no_nm_cb["Spa_iqr75"].astype(int)
df_no_nm_cb["VRDeck_iqr75"] = df_no_nm_cb["VRDeck_iqr75"].astype(int)
df_no_nm_cb["CryoSleep"] = df_no_nm_cb["CryoSleep"].astype(int)
df_no_nm_cb["VIP"]  = df_no_nm_cb["VIP"] .astype(int)
df_no_nm_cb["Transported"] = df_no_nm_cb["Transported"].astype(int)
"""

In [27]:
df_sl_tfdf["Transported"] = df_sl_tfdf["Transported"].astype(int)
df_sl_tfdf["CryoSleep"] = df_sl_tfdf["CryoSleep"].astype(int)

In [28]:
df_train, df_ev = train_test_split(df_sl_tfdf, test_size=0.2, random_state=20)

In [29]:
df_train

,HomePlanet,CryoSleep,Destination,Age,VIP,Transported,Deck,Cabin_num,Side,sl_RS,sl_FC,sl_SM,sl_SPA,sl_VR
1402,Earth,0,PSO J318.5-22,39.0,0.0,0,F,289,P,0.000000,0.311475,33.318182,0.000000,17.525
6488,Earth,0,TRAPPIST-1e,22.0,0.0,0,F,1424,P,17.926829,2.262295,0.000000,0.000000,0.000
6199,Mars,1,TRAPPIST-1e,29.0,0.0,1,F,1362,P,0.000000,0.000000,0.000000,0.000000,0.000
0,Europa,0,TRAPPIST-1e,39.0,0.0,0,B,0,P,0.000000,0.000000,0.000000,0.000000,0.000
6645,Europa,0,TRAPPIST-1e,27.0,0.0,1,E,450,P,0.000000,47.491803,29.954545,9.150943,0.275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5910,Mars,1,TRAPPIST-1e,33.0,0.0,1,F,1202,S,0.000000,0.000000,0.000000,0.000000,0.000
3915,Europa,1,TRAPPIST-1e,44.0,0.0,1,C,133,P,0.000000,0.000000,0.000000,0.000000,0.000
7068,Earth,0,TRAPPIST-1e,41.0,0.0,0,G,1214,P,0.000000,0.000000,0.000000,0.000000,22.700
7391,Earth,0,TRAPPIST-1e,60.0,0.0,0,G,1276,P,0.000000,0.000000,0.000000,0.000000,0.000


In [30]:
ds_no_nm_cb = tfdf.keras.pd_dataframe_to_tf_dataset(df_train, label="Transported")
ds_ev_no_nm_cb = tfdf.keras.pd_dataframe_to_tf_dataset(df_ev, label="Transported")
ds_test = tfdf.keras.pd_dataframe_to_tf_dataset(df_test_sl_tfdf)

In [31]:
df_train

,HomePlanet,CryoSleep,Destination,Age,VIP,Transported,Deck,Cabin_num,Side,sl_RS,sl_FC,sl_SM,sl_SPA,sl_VR
1402,Earth,0,PSO J318.5-22,39.0,0.0,0,F,289,P,0.000000,0.311475,33.318182,0.000000,17.525
6488,Earth,0,TRAPPIST-1e,22.0,0.0,0,F,1424,P,17.926829,2.262295,0.000000,0.000000,0.000
6199,Mars,1,TRAPPIST-1e,29.0,0.0,1,F,1362,P,0.000000,0.000000,0.000000,0.000000,0.000
0,Europa,0,TRAPPIST-1e,39.0,0.0,0,B,0,P,0.000000,0.000000,0.000000,0.000000,0.000
6645,Europa,0,TRAPPIST-1e,27.0,0.0,1,E,450,P,0.000000,47.491803,29.954545,9.150943,0.275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5910,Mars,1,TRAPPIST-1e,33.0,0.0,1,F,1202,S,0.000000,0.000000,0.000000,0.000000,0.000
3915,Europa,1,TRAPPIST-1e,44.0,0.0,1,C,133,P,0.000000,0.000000,0.000000,0.000000,0.000
7068,Earth,0,TRAPPIST-1e,41.0,0.0,0,G,1214,P,0.000000,0.000000,0.000000,0.000000,22.700
7391,Earth,0,TRAPPIST-1e,60.0,0.0,0,G,1276,P,0.000000,0.000000,0.000000,0.000000,0.000


In [99]:
tf.debugging.set_log_device_placement(True)

In [14]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [15]:
tf.debugging.experimental.enable_dump_debug_info("spaceship-titanic/spaceship-titanic/logs/fit", tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

INFO:tensorflow:Enabled dumping callback in thread MainThread (dump root: spaceship-titanic/spaceship-titanic/logs/fit, tensor debug mode: FULL_HEALTH)


INFO:tensorflow:Enabled dumping callback in thread MainThread (dump root: spaceship-titanic/spaceship-titanic/logs/fit, tensor debug mode: FULL_HEALTH)


In [32]:
tf.debugging.set_log_device_placement(True)
tuner = tfdf.tuner.RandomSearch(
    num_trials=100, 
    use_predefined_hps = True,
    trial_num_threads = 1,
    trial_maximum_training_duration_seconds= None
)

arin_rf = tfdf.keras.RandomForestModel(hyperparameter_template="benchmark_rank1")
arin_rf.compile(metrics=["accuracy"])
arin_rf_his = arin_rf.fit(x = ds_no_nm_cb, verbose=2, callbacks=[tensorboard_callback])

print(arin_rf.summary())

Resolve hyper-parameter template "benchmark_rank1" to "benchmark_rank1@v1" -> {'winner_take_all': True, 'categorical_algorithm': 'RANDOM', 'split_axis': 'SPARSE_OBLIQUE', 'sparse_oblique_normalization': 'MIN_MAX', 'sparse_oblique_num_projections_exponent': 1.0}.
Use /tmp/tmpvyvl87ln as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'HomePlanet': <tf.Tensor 'data:0' shape=(None,) dtype=string>, 'CryoSleep': <tf.Tensor 'data_1:0' shape=(None,) dtype=int64>, 'Destination': <tf.Tensor 'data_2:0' shape=(None,) dtype=string>, 'Age': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'VIP': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'Deck': <tf.Tensor 'data_5:0' shape=(None,) dtype=string>, 'Cabin_num': <tf.Tensor 'data_6:0' shape=(None,) dtype=string>, 'Side': <tf.Tensor 'data_7:0' shape=(None,) dtype=string>, 'sl_RS': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'sl_FC': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'sl_S

[INFO 23-11-24 23:49:08.0818 KST kernel.cc:773] Start Yggdrasil model training
[INFO 23-11-24 23:49:08.0818 KST kernel.cc:774] Collect training examples
[INFO 23-11-24 23:49:08.0818 KST kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-11-24 23:49:08.0819 KST kernel.cc:393] Number of batches: 7
[INFO 23-11-24 23:49:08.0819 KST kernel.cc:394] Number of examples: 6954
[INFO 23-11-24 23:49:08.0833 KST data_spec_inference.cc:305] 1279 item(s) have been pruned (i.e. they are considered out of dictionary) for the column Cabin_num (473 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
[INFO 23-11-24 23:49:08.0836 KST data_spec

Model trained in 0:00:12.475425
Compiling model...
Model compiled.
Model: "random_forest_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (13):
	Age
	Cabin_num
	CryoSleep
	Deck
	Destination
	HomePlanet
	Side
	VIP
	sl_FC
	sl_RS
	sl_SM
	sl_SPA
	sl_VR

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.   "CryoSleep"  0.269511 ################
    2.         "Age"  0.194479 #########
    3.       "sl_FC"  0.142204 ####
    4.   "Cabin_num"  0.141284 ####
    5.       "sl_RS"  0.126661 ##
    6.         "VIP"  0.116027 #
    7.        "Deck"  0.115093 #
    8.  "HomePlanet"  0.107793 #
    9.      "sl_SPA"  0.107379 #
   10.       "sl_SM"  0.098914 
   11.  

In [57]:
%tensorboard --logdir spaceship-titanic/spaceship-titanic/logs/fit

SyntaxError: invalid syntax (213534274.py, line 1)

In [64]:
tfdf.model_plotter.plot_model(arin_rf)

'\n<script src="https://d3js.org/d3.v6.min.js"></script>\n<div id="tree_plot_8cb519ad718847109ec5f8383a44aaa7"></div>\n<script>\n/*\n * Copyright 2021 Google LLC.\n * Licensed under the Apache License, Version 2.0 (the "License");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *     https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an "AS IS" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\n/**\n *  Plotting of decision trees generated by TF-DF.\n *\n *  A tree is a recursive structure of node objects.\n *  A node contains one or more of the following components:\n *\n *    - A value: Representing the output of the node. If the node is not a leaf,\

In [17]:
inspector = arin_rf.make_inspector()
inspector.evaluation()
evaluation = arin_rf.evaluate(x=ds_ev_no_nm_cb, return_dict=True)
for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

2/2 [==============================] - 1s 121ms/step - loss: 0.0000e+00 - accuracy: 0.8016
loss: 0.0000
accuracy: 0.8016


In [32]:
for importance in inspector.variable_importances().keys():
  print("\t", importance)
inspector.variable_importances()["NUM_AS_ROOT"]

	 INV_MEAN_MIN_DEPTH
	 NUM_AS_ROOT
	 SUM_SCORE
	 NUM_NODES


[("CryoSleep" (1; #1), 163.0),
 ("Age" (1; #0), 43.0),
 ("RoomService" (1; #6), 33.0),
 ("FoodCourt" (1; #3), 28.0),
 ("RoomService_iqr75" (1; #7), 18.0),
 ("FoodCourt_iqr75" (1; #4), 3.0),
 ("ShoppingMall" (1; #8), 3.0),
 ("Spa_iqr75" (1; #12), 3.0),
 ("ShoppingMall_iqr75" (1; #9), 2.0),
 ("Spa" (1; #11), 2.0),
 ("VRDeck" (1; #14), 2.0)]

In [18]:
predictions = arin_rf.predict(ds_test)
submission_id = sub["PassengerId"]
n_predictions = (predictions > 0.5).astype(bool)
output = pd.DataFrame({'PassengerId': submission_id,
                       'Transported': n_predictions.squeeze()})


5/5 [==============================] - 1s 77ms/step


In [19]:
output.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


In [20]:

sub['Transported'] = n_predictions
sub.to_csv('submission.csv', index=False)
sub.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
